In [30]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, datasets, linear_model, model_selection
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from cmath import sqrt

dataframe=pd.read_csv('../Dataset.csv')

Видалимо всі рядки з порожніми даними, та закодуємо стовпчик з фічею 'platform'.

In [31]:
dataframe.drop(dataframe.index[dataframe.isna().any(axis=1)].tolist(),axis=0,inplace=True)
income_dataframe=dataframe.filter(['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1)
dataframe.drop(columns=['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1,inplace=True)

dataframe.drop(columns=['media_source','install_date','country_code'],axis=1,inplace=True)
dummies_platform=pd.get_dummies(dataframe['platform'],drop_first=True)
dataframe.drop(columns=['platform'],axis=1,inplace=True)
dataframe=pd.concat([dataframe,dummies_platform],axis='columns')
print(dataframe)

         total_sessions_day0  total_sessions_day1  total_sessions_day3  \
0                        1.0                  1.0                  1.0   
1                        1.0                  1.0                  1.0   
2                        1.0                  2.0                  2.0   
3                        1.0                  1.0                  1.0   
4                        1.0                  1.0                  1.0   
...                      ...                  ...                  ...   
1467827                 11.0                 11.0                 11.0   
1467828                  2.0                  3.0                  6.0   
1467829                  3.0                  5.0                 11.0   
1467830                  5.0                  5.0                  5.0   
1467831                  6.0                  7.0                  7.0   

         total_sessions_day7  chapters_finished_day0  chapters_finished_day1  \
0                        1.0   

In [32]:
new_dataframe=dataframe.copy()

Натренуємо модель на всіх фічах для того, щоб на основі абсолютних значень коефіцієнтів впорядкувати фічі за "важливістю".

In [33]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(new_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)
model.fit(X_train, y_train)
coef=[]
for i in range(len(model.coef_)):
    coef.append((new_dataframe.columns.values[i], abs(model.coef_[i])))
coef=sorted(coef, key=lambda tup: tup[1])
coef.reverse()
for i in range(len(model.coef_)):
    print(coef[i])

('ad_ltv_day3', 4.138487240324049)
('ad_ltv_day1', 2.3954567511554776)
('app_sub_ltv_day0', 1.6542535345766984)
('app_iap_ltv_day3', 1.6503985258844596)
('app_sub_ltv_day3', 1.3430796076966214)
('app_iap_ltv_day1', 0.44649850555256165)
('retained_day7', 0.3422940782452222)
('ad_ltv_day0', 0.3206918401705674)
('app_sub_ltv_day1', 0.28288525965363787)
('retained_day3', 0.22030077963536432)
('chapters_closed_day7', 0.15356161648287267)
('chapters_closed_day3', 0.15354522095025425)
('chapters_finished_day3', 0.12423537826861854)
('app_iap_ltv_day0', 0.11389683751423396)
('retained_day1', 0.09936317404386785)
('chapters_finished_day7', 0.08322799973206771)
('chapters_opened_session9', 0.07376704764743192)
('chapters_closed_session9', 0.0687884914536015)
('tickets_spent_day1', 0.06223477733744215)
('chapters_finished_session9', 0.050564844777816254)
('chapters_opened_day3', 0.04987499743880077)
('tickets_spent_day3', 0.04498298970091197)
('tickets_spent_session1', 0.040955433564909716)
('cha

Будемо поступово(за "важливістю") додавати фічі в модель і у випадку покращення метрики MAE залишати їх в моделі.

In [34]:
features=[]
min=1000000
best_features=[]
for x in coef:
    name=x[1]
    features.append(x[0])
    training_dataframe=new_dataframe.filter(features,axis='columns')
    X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
    model=linear_model.LinearRegression(fit_intercept=1)    
    model.fit(X_train, y_train)
    predictions=model.predict(X_test)
    a=mean_absolute_error(predictions,y_test)
    if a<min:
        best_features=features.copy()
        min=a
        print(x[0],a)
    else:
        features.remove(x[0])

    


ad_ltv_day3 0.3786347271718978
app_sub_ltv_day0 0.3730327367040168
app_iap_ltv_day3 0.14829566492855661
app_sub_ltv_day3 0.13828281671087436
app_iap_ltv_day1 0.12145577251342274
chapters_finished_day3 0.12047540779073365
retained_day1 0.11863352139132247


In [35]:
print(best_features,min)

['ad_ltv_day3', 'app_sub_ltv_day0', 'app_iap_ltv_day3', 'app_sub_ltv_day3', 'app_iap_ltv_day1', 'chapters_finished_day3', 'retained_day1'] 0.11863352139132247


In [36]:
training_dataframe=new_dataframe.filter(best_features,axis='columns')
X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)    
model.fit(X_train, y_train)
predictions=model.predict(X_test)
print(mean_absolute_error(predictions,y_test))
print(sqrt(mean_squared_error(predictions,y_test)))
print(mean_absolute_percentage_error(predictions,y_test))

0.1229251304688254
(1.427245448896613+0j)
7.600582193367486


Водночас бачимо, що модель з фічами 'ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3' має майже таку ж метрику MAE. Враховуючи той факт, що фічі 'ad_ltv_day3' та 'ad_ltv_day1' корелюють (як видно з EDA) то можна спробувати викинути одну з них. Викинемо 'ad_ltv_day1', адже вона має меншу "важливість".

In [37]:
training_dataframe=new_dataframe.filter(['ad_ltv_day3','app_iap_ltv_day3'],axis='columns')
X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)    
model.fit(X_train, y_train)
predictions=model.predict(X_test)
print(mean_absolute_error(predictions,y_test))
print(sqrt(mean_squared_error(predictions,y_test)))
print(mean_absolute_percentage_error(predictions,y_test))

0.14099818976182668
(1.210911514926889+0j)
2.300020927213265


Бачимо що метрика MAE не набагато більша за модель з best_features тому виберемо простішу модель.

In [38]:
print(model.coef_)
print(model.intercept_)

[3.27839123 1.48129353]
-0.008201597945691586
